In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY_1')

### Suppose today is 03/30/25 Calculate Frequency and Value (RFV)

# Task
Generate Python code to calculate RFV (Recency, Frequency, Monetary Value) for customers using the dataset "rfv.xls", assuming today's date is 30/03/2025. Include English and Portuguese hashtags in the code comments.

## Load data / carregar dados

### Subtask:
Load the data from the 'rfv.xlsx' file into a pandas DataFrame. / Carregar os dados do arquivo 'rfv.xlsx' em um DataFrame pandas.


**Reasoning**:
Import pandas, load the data from the Excel file, and display the head and info of the DataFrame to understand its structure.



In [ ]:
import pandas as pd # Importar a biblioteca pandas

# Read the Excel file named 'rfv.xlsx' into a pandas DataFrame.
# Ler o arquivo Excel chamado 'rfv.xlsx' em um DataFrame pandas
df = pd.read_excel('/content/rfv.xlsx')

# Display the first 5 rows of the DataFrame and its information to understand its structure.
# Exibir as primeiras 5 linhas do DataFrame e suas informações para entender sua estrutura.
display(df.head())
display(df.info())

## Data preparation / preparação dos dados

### Subtask:
Convert the transaction date column to datetime format and calculate the recency for each customer. / Converter a coluna de data da transação para o formato datetime e calcular a recência para cada cliente.


**Reasoning**:
Rename columns, convert date column to datetime, define today's date, and calculate recency for each customer.



In [ ]:
# Rename columns for clarity. # Renomear colunas para clareza.
df = df.rename(columns={'CLIEBNTE': 'Cliente', 'Data_Compra': 'Data'})

# Convert 'Data' column to datetime objects if it's not already. # Converter a coluna 'Data' para objetos datetime, caso ainda não seja.
df['Data'] = pd.to_datetime(df['Data'])

# Define today's date. # Definir a data de hoje.
today_date = pd.to_datetime('2025-03-30')

# Calculate recency for each customer. # Calcular recência para cada cliente.
# Find the most recent purchase date for each customer. # Encontrar a data de compra mais recente para cada cliente.
most_recent_purchase = df.groupby('Cliente')['Data'].max().reset_index()

# Calculate the difference in days between today's date and the most recent purchase date. # Calcular a diferença em dias entre a data de hoje e a data de compra mais recente.
most_recent_purchase['Recency'] = (today_date - most_recent_purchase['Data']).dt.days

# Display the recency for each customer. # Exibir a recência para cada cliente.
# Display the columns Cliente, Data, and Recency as shown in the attached image.
# Exibir as colunas Cliente, Data e Recency conforme mostrado na imagem anexa.
display(most_recent_purchase[['Cliente', 'Data', 'Recency']].head())

## Calculate rfv / calcular rfv

### Subtask:
Calculate Frequency and Monetary Value for each customer. / Calcular Frequência e Valor Monetário para cada cliente.


**Reasoning**:
Calculate the frequency and monetary value for each customer and then merge these with the previously calculated recency data.



In [ ]:
# Calculate Frequency for each customer by counting transactions. # Calcular a Frequência para cada cliente contando as transações.
frequency_df = df.groupby('Cliente').size().reset_index(name='Frequency')

# Calculate Monetary Value for each customer by summing 'Valor'. # Calcular o Valor Monetário para cada cliente somando 'Valor'.
monetary_df = df.groupby('Cliente')['Valor'].sum().reset_index(name='MonetaryValue')

# Merge Recency, Frequency, and Monetary Value data. # Combinar dados de Recência, Frequência e Valor Monetário.
rfv_df = most_recent_purchase.merge(frequency_df, on='Cliente')
rfv_df = rfv_df.merge(monetary_df, on='Cliente')

# Display the head of the resulting RFV DataFrame. # Exibir o cabeçalho do DataFrame RFV resultante.
display(rfv_df.head())

## Rfv scoring / pontuação rfv

### Subtask:
Assign RFV scores to customers based on their Recency, Frequency, and Monetary Value. / Atribuir pontuações RFV aos clientes com base em sua Recência, Frequência e Valor Monetário.


**Reasoning**:
Assign RFV scores by creating quartiles for Recency, Frequency, and Monetary Value and then combine these scores.



In [ ]:
# Assign R, F, and M scores based on quartiles. # Atribuir pontuações R, F e M com base em quartis.
# Recency: Higher recency (more days) gets a lower score. # Recência: Recência maior (mais dias) obtém pontuação menor.
rfv_df['R_score'] = pd.qcut(rfv_df['Recency'], 4, labels=[4, 3, 2, 1])
# Frequency: Higher frequency gets a higher score. # Frequência: Frequência maior obtém pontuação maior.
rfv_df['F_score'] = pd.qcut(rfv_df['Frequency'], 4, labels=[1, 2, 3, 4])
# MonetaryValue: Higher monetary value gets a higher score. # Valor Monetário: Valor monetário maior obtém pontuação maior.
rfv_df['M_score'] = pd.qcut(rfv_df['MonetaryValue'], 4, labels=[1, 2, 3, 4])

# Combine R, F, and M scores to create a single RFM score. # Combinar pontuações R, F e M para criar uma única pontuação RFM.
# Concatenate the string representations of the scores. # Concatenar as representações de string das pontuações.
rfv_df['RFM_score'] = rfv_df['R_score'].astype(str) + rfv_df['F_score'].astype(str) + rfv_df['M_score'].astype(str)

# Display the head of the DataFrame with the new scores. # Exibir o cabeçalho do DataFrame com as novas pontuações.
display(rfv_df.head())

## Analyze rfv segments / analisar segmentos rfv

### Subtask:
Segment customers based on their RFV scores. / Segmentar clientes com base em suas pontuações RFV.


**Reasoning**:
Define RFM segments based on RFM scores and apply this segmentation to the dataframe, then display the counts and average RFV metrics per segment.



In [ ]:
# Define RFM segments. # Definir segmentos RFM.
# This is a sample segmentation strategy, you can adjust it based on your business needs.
# Esta é uma estratégia de segmentação de amostra, você pode ajustá-la com base nas necessidades do seu negócio.
segment_map = {
    r'[4-5][4-5][4-5]': 'Champions', # Customers who buy recently, frequently, and spend the most. # Clientes que compram recentemente, com frequência e gastam mais.
    r'[3-4][3-4][3-4]': 'Loyal Customers', # Customers who buy often and spend a good amount. # Clientes que compram frequentemente e gastam uma boa quantia.
    r'[2-3][3-4][3-4]': 'Potential Loyalists', # Recent customers with average frequency and monetary value. # Clientes recentes com frequência e valor monetário médios.
    r'[4-5]1[1-2]': 'New Customers', # Customers who bought recently but not frequently. # Clientes que compraram recentemente, mas não com frequência.
    r'[1-2][3-4][3-4]': 'At Risk', # Customers who haven't bought recently but bought frequently and spent a lot. # Clientes que não compraram recentemente, mas compraram com frequência e gastaram muito.
    r'[1-2]2[3-4]': 'Cant Lose Them', # Customers who haven't bought recently but spent a lot. # Clientes que não compraram recentemente, mas gastaram muito.
    r'[1-2][1-2][1-2]': 'Lost', # Customers who haven't bought recently, not frequent and didn't spend much. # Clientes que não compraram recentemente, não são frequentes e não gastaram muito.
}

# Apply the segmentation based on the RFM_score. # Aplicar a segmentação com base na pontuação RFM.
rfv_df['RFM_Segment'] = rfv_df['RFM_score'].replace(segment_map, regex=True)

# Display the count of customers in each segment. # Exibir a contagem de clientes em cada segmento.
display(rfv_df['RFM_Segment'].value_counts())

# Display the average Recency, Frequency, and Monetary Value for each segment. # Exibir a média de Recência, Frequência e Valor Monetário para cada segmento.
display(rfv_df.groupby('RFM_Segment')[['Recency', 'Frequency', 'MonetaryValue']].mean())

## Summary:

### Data Analysis Key Findings

*   The dataset contains 10000 entries with columns for customer identifier, purchase date, and transaction value.
*   The analysis successfully calculated Recency (days since last purchase), Frequency (number of transactions), and Monetary Value (total spent) for each customer.
*   RFV scores were assigned based on the quartiles of Recency, Frequency, and Monetary Value, with Recency scored in reverse order.
*   Customers were segmented into categories like 'Champions', 'Loyal Customers', 'Lost', etc., based on their combined RFM scores.
*   The distribution of customers across segments was determined, and the average RFV metrics for each segment were calculated, providing insights into the characteristics of each group.

### Insights or Next Steps

*   Analyze the characteristics of each RFM segment in more detail to develop targeted marketing strategies.
*   Consider refining the segmentation strategy based on business-specific knowledge and goals.


## Visualize RFV / Visualizar RFV

### Subtask:
Generate visualizations for Recency, Frequency, Monetary Value, and RFM segments using a dark mode and turquoise color scheme. / Gerar visualizações para Recência, Frequência, Valor Monetário e segmentos RFM usando um modo escuro e esquema de cores turquesa.

**Reasoning**:
Generate histograms for Recency, Frequency, and Monetary Value to visualize their distributions, and a bar plot for the RFM segments to see the customer distribution across segments. Use matplotlib and seaborn with a dark mode style and a turquoise color palette.

In [ ]:
import matplotlib.pyplot as plt # Importar a biblioteca de plotagem
import seaborn as sns # Importar a biblioteca de visualização de dados
import numpy as np # Importar a biblioteca numpy

# Set dark mode style for plots # Configurar o estilo de modo escuro para os gráficos
plt.style.use('dark_background')

# Define a turquoise-based color palette with enough colors for all segments # Definir uma paleta de cores baseada em turquesa com cores suficientes para todos os segmentos
# Get the number of unique segments
num_segments = rfv_df['RFM_Segment'].nunique()
# Use a palette that can provide enough distinct colors, e.g., 'viridis' or 'plasma'
# We can also create a custom palette or use a larger built-in palette if needed.
# For simplicity and to ensure enough colors, let's use a larger palette or a different approach.

# Let's use a different approach for the RFM segment plot to ensure enough colors.
# We can map each segment to a color from a suitable colormap.
colors = plt.cm.viridis(np.linspace(0, 1, num_segments))
segment_color_map = {segment: colors[i] for i, segment in enumerate(rfv_df['RFM_Segment'].unique())}


# Visualize the distribution of Recency # Visualizar a distribuição da Recência
plt.figure(figsize=(10, 6))
sns.histplot(rfv_df['Recency'], bins=50, kde=True, color='turquoise') # Usando um tom de turquesa
plt.title('Distribution of Recency / Distribuição da Recência') # Título do gráfico
plt.xlabel('Recency (Days) / Recência (Dias)') # Rótulo do eixo x
plt.ylabel('Number of Customers / Número de Clientes') # Rótulo do eixo y
plt.show()

# Visualize the distribution of Frequency # Visualizar a distribuição da Frequência
plt.figure(figsize=(10, 6))
sns.histplot(rfv_df['Frequency'], bins=50, kde=True, color='teal') # Usando um tom de turquesa
plt.title('Distribution of Frequency / Distribuição da Frequência') # Título do gráfico
plt.xlabel('Frequency (Number of Transactions) / Frequência (Número de Transações)') # Rótulo do eixo x
plt.ylabel('Number of Customers / Número de Clientes') # Rótulo do eixo y
plt.show()

# Visualize the distribution of Monetary Value # Visualizar a distribuição do Valor Monetário
plt.figure(figsize=(10, 6))
sns.histplot(rfv_df['MonetaryValue'], bins=50, kde=True, color='darkcyan') # Usando um tom de turquesa
plt.title('Distribution of Monetary Value / Distribuição do Valor Monetário') # Título do gráfico
plt.xlabel('Monetary Value / Valor Monetário') # Rótulo do eixo x
plt.ylabel('Number of Customers / Número de Clientes') # Rótulo do eixo y
plt.show()

# Visualize the distribution of RFM Segments # Visualizar a distribuição dos Segmentos RFM
plt.figure(figsize=(15, 7)) # Increased figure size for better readability
sns.countplot(data=rfv_df, x='RFM_Segment', palette=segment_color_map) # Usando o mapa de cores para os segmentos
plt.title('Distribution of RFM Segments / Distribuição dos Segmentos RFM') # Título do gráfico
plt.xlabel('RFM Segment / Segmento RFM') # Rótulo do eixo x
plt.ylabel('Number of Customers / Número de Clientes') # Rótulo do eixo y
plt.xticks(rotation=45, ha='right') # Rotacionar rótulos do eixo x para melhor legibilidade
plt.tight_layout() # Ajustar layout para evitar sobreposição
plt.show()

**Reasoning**:
Re-calculate the RFM segments as the previous execution failed to find the `RFM_Segment` column.

In [ ]:
# Define RFM segments. # Definir segmentos RFM.
# This is a sample segmentation strategy, you can adjust it based on your business needs.
# Esta é uma estratégia de segmentação de amostra, você pode ajustá-la com base nas necessidades do seu negócio.
segment_map = {
    r'[4-5][4-5][4-5]': 'Champions', # Customers who buy recently, frequently, and spend the most. # Clientes que compram recentemente, com frequência e gastam mais.
    r'[3-4][3-4][3-4]': 'Loyal Customers', # Customers who buy often and spend a good amount. # Clientes que compram frequentemente e gastam uma boa quantia.
    r'[2-3][3-4][3-4]': 'Potential Loyalists', # Recent customers with average frequency and monetary value. # Clientes recentes com frequência e valor monetário médios.
    r'[4-5]1[1-2]': 'New Customers', # Customers who bought recently but not frequently. # Clientes que compraram recentemente, mas não com frequência.
    r'[1-2][3-4][3-4]': 'At Risk', # Customers who haven't bought recently but bought frequently and spent a lot. # Clientes que não compraram recentemente, mas compraram com frequência e gastaram muito.
    r'[1-2]2[3-4]': 'Cant Lose Them', # Customers who haven't bought recently but spent a lot. # Clientes que não compraram recentemente, mas gastaram muito.
    r'[1-2][1-2][1-2]': 'Lost', # Customers who haven't bought recently, not frequent and didn't spend much. # Clientes que não compraram recentemente, não são frequentes e não gastaram muito.
}

# Apply the segmentation based on the RFM_score. # Aplicar a segmentação com base na pontuação RFM.
rfv_df['RFM_Segment'] = rfv_df['RFM_score'].replace(segment_map, regex=True)

# Display the count of customers in each segment. # Exibir a contagem de clientes em cada segmento.
display(rfv_df['RFM_Segment'].value_counts())

# Display the average Recency, Frequency, and Monetary Value for each segment. # Exibir a média de Recência, Frequência e Valor Monetário para cada segmento.
display(rfv_df.groupby('RFM_Segment')[['Recency', 'Frequency', 'MonetaryValue']].mean())

**Reasoning**:
Re-calculate the RFV scores and segments as the previous executions failed to find the necessary columns.

In [ ]:
# Assign R, F, and M scores based on quartiles. # Atribuir pontuações R, F e M com base em quartis.
# Recency: Higher recency (more days) gets a lower score. # Recência: Recência maior (mais dias) obtém pontuação menor.
rfv_df['R_score'] = pd.qcut(rfv_df['Recency'], 4, labels=[4, 3, 2, 1])
# Frequency: Higher frequency gets a higher score. # Frequência: Frequência maior obtém pontuação maior.
rfv_df['F_score'] = pd.qcut(rfv_df['Frequency'], 4, labels=[1, 2, 3, 4])
# MonetaryValue: Higher monetary value gets a higher score. # Valor monetário: Valor monetário maior obtém pontuação maior.
rfv_df['M_score'] = pd.qcut(rfv_df['MonetaryValue'], 4, labels=[1, 2, 3, 4])

# Combine R, F, and M scores to create a single RFM score. # Combinar pontuações R, F e M para criar uma única pontuação RFM.
# Concatenate the string representations of the scores. # Concatenar as representações de string das pontuações.
rfv_df['RFM_score'] = rfv_df['R_score'].astype(str) + rfv_df['F_score'].astype(str) + rfv_df['M_score'].astype(str)

# Define RFM segments. # Definir segmentos RFM.
# This is a sample segmentation strategy, you can adjust it based on your business needs.
# Esta é uma estratégia de segmentação de amostra, você pode ajustá-la com base nas necessidades do seu negócio.
segment_map = {
    r'[4-5][4-5][4-5]': 'Champions', # Customers who buy recently, frequently, and spend the most. # Clientes que compram recentemente, com frequência e gastam mais.
    r'[3-4][3-4][3-4]': 'Loyal Customers', # Customers who buy often and spend a good amount. # Clientes que compram frequentemente e gastam uma boa quantia.
    r'[2-3][3-4][3-4]': 'Potential Loyalists', # Recent customers with average frequency and monetary value. # Clientes recentes com frequência e valor monetário médios.
    r'[4-5]1[1-2]': 'New Customers', # Customers who bought recently but not frequently. # Clientes que compraram recentemente, mas não com frequência.
    r'[1-2][3-4][3-4]': 'At Risk', # Customers who haven't bought recently but bought frequently and spent a lot. # Clientes que não compraram recentemente, mas compraram com frequência e gastaram muito.
    r'[1-2]2[3-4]': 'Cant Lose Them', # Customers who haven't bought recently but spent a lot. # Clientes que não compraram recentemente, mas gastaram muito.
    r'[1-2][1-2][1-2]': 'Lost', # Customers who haven't bought recently, not frequent and didn't spend much. # Clientes que não compraram recentemente, não são frequentes e não gastaram muito.
}

# Apply the segmentation based on the RFM_score. # Aplicar a segmentação com base na pontuação RFM.
rfv_df['RFM_Segment'] = rfv_df['RFM_score'].replace(segment_map, regex=True)

# Display the head of the DataFrame with the new scores and segments. # Exibir o cabeçalho do DataFrame com as novas pontuações e segmentos.
display(rfv_df.head())

# Display the count of customers in each segment. # Exibir a contagem de clientes em cada segmento.
display(rfv_df['RFM_Segment'].value_counts())

# Display the average Recency, Frequency, and Monetary Value for each segment. # Exibir a média de Recência, Frequência e Valor Monetário para cada segmento.
display(rfv_df.groupby('RFM_Segment')[['Recency', 'Frequency', 'MonetaryValue']].mean())

## Summary:

### Data Analysis Key Findings

* The dataset contains 10000 entries with columns for customer identifier, purchase date, and transaction value.
* The analysis successfully calculated Recency (days since last purchase), Frequency (number of transactions), and Monetary Value (total spent) for each customer.
* RFV scores were assigned based on the quartiles of Recency, Frequency, and Monetary Value, with Recency scored in reverse order.
* Customers were segmented into categories like 'Champions', 'Loyal Customers', 'Lost', etc., based on their combined RFM scores.
* The distribution of customers across segments was determined, and the average RFV metrics for each segment were calculated, providing insights into the characteristics of each group.

### Insights or Next Steps

* Analyze the characteristics of each RFM segment in more detail to develop targeted marketing strategies.
* Consider refining the segmentation strategy based on business-specific knowledge and goals.

## Final DataFrame Structure / Estrutura Final do DataFrame

In [ ]:
# Display the head of the final DataFrame with all calculated columns.
# Exibir o cabeçalho do DataFrame final com todas as colunas calculadas.
display(rfv_df.head())

## Summary:

### Data Analysis Key Findings

* The dataset contains 10000 entries with columns for customer identifier, purchase date, and transaction value.
* The analysis successfully calculated Recency (days since last purchase), Frequency (number of transactions), and Monetary Value (total spent) for each customer.
* RFV scores were assigned based on the quartiles of Recency, Frequency, and Monetary Value, with Recency scored in reverse order.
* Customers were segmented into categories like 'Champions', 'Loyal Customers', 'Lost', etc., based on their combined RFM scores.
* The distribution of customers across segments was determined, and the average RFV metrics for each segment were calculated, providing insights into the characteristics of each group.

### Insights or Next Steps

* Analyze the characteristics of each RFM segment in more detail to develop targeted marketing strategies.
* Consider refining the segmentation strategy based on business-specific knowledge and goals.

## Verify data counts / Verificar contagens de dados

### Subtask:
Verify the number of transactions in the original DataFrame and the number of unique customers. / Verificar o número de transações no DataFrame original e o número de clientes únicos.

**Reasoning**:
Display the total number of rows in the original DataFrame to confirm the transaction count and the number of unique values in the 'Cliente' column of the RFV DataFrame to confirm the customer count.

In [ ]:
# Verify the number of transactions # Verificar o número de transações
print(f"Number of transactions (rows in original DataFrame): {len(df)}") # Número de transações (linhas no DataFrame original)

# Verify the number of unique customers # Verificar o número de clientes únicos
print(f"Number of unique customers: {rfv_df['Cliente'].nunique()}") # Número de clientes únicos

## Export RFV Results to Excel / Exportar resultados RFV para Excel

### Subtask:
Export the DataFrame containing the RFV analysis results to an Excel file. / Exportar o DataFrame contendo os resultados da análise RFV para um arquivo Excel.

**Reasoning**:
Export the `rfv_df` DataFrame to an Excel file using the `to_excel()` method.

In [ ]:
# Sort the DataFrame by RFM_score in descending order before exporting. # Ordenar o DataFrame pela pontuação RFM em ordem decrescente antes de exportar.
rfv_df_sorted = rfv_df.sort_values(by='RFM_score', ascending=False)

# Export the sorted rfv_df DataFrame to an Excel file. # Exportar o DataFrame rfv_df ordenado para um arquivo Excel.
rfv_df_sorted.to_excel('rfv_results_sorted.xlsx', index=False) # Salvar o DataFrame ordenado como 'rfv_results_sorted.xlsx' sem o índice.

print("RFV results exported to 'rfv_results_sorted.xlsx' in descending order by RFM score / Resultados RFV exportados para 'rfv_results_sorted.xlsx' em ordem decrescente pela pontuação RFM") # Mensagem de confirmação.